---
layout: post
title: CSA Google Maps
description: Google Maps knockoff
type: issues 
comments: true
permalink: /csa-gm/
---

<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Pathfinding Visualizer</title>
  <style>
    canvas { border: 1px solid #000; }
  </style>
</head>
<body>
  <button onclick="runAStar()">Run A*</button>
  <button onclick="runDijkstra()">Run Dijkstra</button>
  <canvas id="grid" width="320" height="280"></canvas>

  <script>
    const rows = 7, cols = 8;
    const grid = [
      [0,0,0,0,1,0,0,0],
      [1,1,0,0,1,0,1,0],
      [0,0,0,1,0,0,1,0],
      [0,1,1,0,0,1,0,0],
      [0,0,0,0,0,0,0,1],
      [1,0,1,1,0,1,0,0],
      [0,0,0,0,0,0,0,0]
    ];
    const start = { x: 0, y: 0 };
    const goal = { x: 7, y: 6 };

    const canvas = document.getElementById('grid');
    const ctx = canvas.getContext('2d');
    const cellSize = 40;

    function drawGrid(path = []) {
      for (let y = 0; y < rows; y++) {
        for (let x = 0; x < cols; x++) {
          ctx.fillStyle = grid[y][x] === 1 ? '#444' : '#fff';
          ctx.fillRect(x * cellSize, y * cellSize, cellSize, cellSize);
          ctx.strokeRect(x * cellSize, y * cellSize, cellSize, cellSize);
        }
      }
      for (const { x, y } of path) {
        ctx.fillStyle = 'lightgreen';
        ctx.fillRect(x * cellSize, y * cellSize, cellSize, cellSize);
      }
      ctx.fillStyle = 'blue';
      ctx.fillRect(start.x * cellSize, start.y * cellSize, cellSize, cellSize);
      ctx.fillStyle = 'red';
      ctx.fillRect(goal.x * cellSize, goal.y * cellSize, cellSize, cellSize);
    }

    function heuristic(a, b) {
      return Math.abs(a.x - b.x) + Math.abs(a.y - b.y);
    }

    function reconstructPath(cameFrom, current) {
      const path = [];
      while (`${current.x},${current.y}` in cameFrom) {
        path.push(current);
        current = cameFrom[`${current.x},${current.y}`];
      }
      path.push(start);
      drawGrid(path.reverse());
    }

    function runAStar() {
      const openSet = [start];
      const cameFrom = {};
      const gScore = Array(rows).fill(0).map(() => Array(cols).fill(Infinity));
      const fScore = Array(rows).fill(0).map(() => Array(cols).fill(Infinity));
      gScore[start.y][start.x] = 0;
      fScore[start.y][start.x] = heuristic(start, goal);

      while (openSet.length > 0) {
        openSet.sort((a, b) => fScore[a.y][a.x] - fScore[b.y][b.x]);
        const current = openSet.shift();
        if (current.x === goal.x && current.y === goal.y) {
          return reconstructPath(cameFrom, current);
        }

        for (const [dx, dy] of [[0,1],[1,0],[0,-1],[-1,0]]) {
          const nx = current.x + dx, ny = current.y + dy;
          if (nx >= 0 && nx < cols && ny >= 0 && ny < rows && grid[ny][nx] === 0) {
            const tentativeG = gScore[current.y][current.x] + 1;
            if (tentativeG < gScore[ny][nx]) {
              cameFrom[`${nx},${ny}`] = current;
              gScore[ny][nx] = tentativeG;
              fScore[ny][nx] = tentativeG + heuristic({ x: nx, y: ny }, goal);
              if (!openSet.some(n => n.x === nx && n.y === ny)) {
                openSet.push({ x: nx, y: ny });
              }
            }
          }
        }
      }
      alert("No path found");
    }

    function runDijkstra() {
      const openSet = [start];
      const cameFrom = {};
      const dist = Array(rows).fill(0).map(() => Array(cols).fill(Infinity));
      dist[start.y][start.x] = 0;

      while (openSet.length > 0) {
        openSet.sort((a, b) => dist[a.y][a.x] - dist[b.y][b.x]);
        const current = openSet.shift();
        if (current.x === goal.x && current.y === goal.y) {
          return reconstructPath(cameFrom, current);
        }

        for (const [dx, dy] of [[0,1],[1,0],[0,-1],[-1,0]]) {
          const nx = current.x + dx, ny = current.y + dy;
          if (nx >= 0 && nx < cols && ny >= 0 && ny < rows && grid[ny][nx] === 0) {
            const alt = dist[current.y][current.x] + 1;
            if (alt < dist[ny][nx]) {
              cameFrom[`${nx},${ny}`] = current;
              dist[ny][nx] = alt;
              if (!openSet.some(n => n.x === nx && n.y === ny)) {
                openSet.push({ x: nx, y: ny });
              }
            }
          }
        }
      }
      alert("No path found");
    }

    drawGrid();
  </script>
</body>
</html>
